In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv
/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv


In [2]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
import os
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("dark") # Theme for plots as Dark
# sns.set_palette("viridis")
sns.color_palette("flare")
import tensorflow as tf
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import optuna
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from catboost import Pool, CatBoostClassifier, cv

In [3]:
train_essays1 = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")

In [4]:
train_essays2 = pd.read_csv("/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv")
train_essays3 = pd.read_csv("/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv")
test_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")

test_essays.drop(["prompt_id"], axis=1, inplace=True)
train_essays1.drop(["prompt_id"], axis=1, inplace=True)
train_essays2.drop(["prompt_id"], axis=1, inplace=True)
train_essays4 = train_essays3[["id", "source_text"]]
train_essays4["generated"] = 1
train_essays4.columns = ["id", "text", "generated"]
train_essays3 = train_essays3[["id", "text"]]
train_essays3["generated"] = 0

train_essays = pd.concat([train_essays1, train_essays2, train_essays3, train_essays4])
train_essays.reset_index(drop=True, inplace=True)
train_essays.head(10)

,id,text,generated
0,0059830c,Cars. Cars have been around since they became ...,0
1,005db917,Transportation is a large necessity in most co...,0
2,008f63e3,"""America's love affair with it's vehicles seem...",0
3,00940276,How often do you ride in a car? Do you drive a...,0
4,00c39458,Cars are a wonderful thing. They are perhaps o...,0
5,00da8c32,The electrol college system is an unfair syste...,0
6,011dc2bc,"Dear state senator, It is the utmost respect t...",0
7,01448434,"Fellow citizens, cars have become a major role...",0
8,01c6e176,"""It's official: The electoral college is unfai...",0
9,0202ddf9,The Electoral College has been kept for centur...,0


In [5]:
test_essays

,id,text
0,0000aaaa,Aaa bbb ccc.
1,1111bbbb,Bbb ccc ddd.
2,2222cccc,CCC ddd eee.


In [6]:
VOCAB_SIZE = 10000
MAXLEN = 500

In [7]:
train_essays["oneHot"] = [tf.keras.preprocessing.text.one_hot(i, VOCAB_SIZE) for i in train_essays["text"]]
df = pd.DataFrame(tf.keras.utils.pad_sequences(train_essays["oneHot"], padding="pre", maxlen=MAXLEN))
train_essays = pd.concat((train_essays, df), axis=1)
train_essays.drop(["text", "oneHot"], inplace=True, axis=1)
train_essays.set_index("id", inplace=True)
train_essays.head(10)

,generated,0,1,2,3,4,5,6,7,8,...,490,491,492,493,494,495,496,497,498,499
id,,,,,,,,,,,,,,,,,,,,,
0059830c,0,4282,6543,1596,6171,8524,9408,8956,5077,2140,...,1244,1596,3509,4145,6019,2869,2877,7280,5482,7027
005db917,0,0,0,0,0,0,0,0,0,0,...,8486,3770,4282,5999,1596,4282,6201,4565,6694,4023
008f63e3,0,1941,1591,7246,3827,5845,7945,5357,5482,7027,...,8905,4371,4081,4309,4636,4717,6136,4886,8955,4549
00940276,0,5906,2869,2233,259,729,5978,2609,8602,3177,...,5482,9308,2501,4565,5099,7876,4176,4509,6136,102
00c39458,0,4517,9121,9243,729,5578,4517,9121,3400,4282,...,3569,4371,5825,7913,2882,3400,8986,513,2634,3387
00da8c32,0,3124,8833,6268,3734,4565,262,5566,2869,3387,...,6106,8486,6136,6548,2145,4282,1343,9797,3836,7580
011dc2bc,0,0,0,0,0,0,0,0,0,0,...,3227,9222,9233,8033,9140,8955,4282,2765,6136,7691
01448434,0,0,0,0,0,0,0,0,0,0,...,7518,6091,2535,35,4565,9243,267,5845,9837,6694
01c6e176,0,0,0,0,0,0,0,0,0,0,...,3836,9536,6136,5488,2958,4565,5574,4889,3374,7172


In [8]:
test_essays["oneHot"] = [tf.keras.preprocessing.text.one_hot(i, VOCAB_SIZE) for i in test_essays["text"]]
df = pd.DataFrame(tf.keras.utils.pad_sequences(test_essays["oneHot"], padding="pre", maxlen=MAXLEN))
test_essays = pd.concat((test_essays,df), axis=1)
test_essays.drop(["text", "oneHot"], inplace=True, axis=1)
test_essays.set_index("id", inplace=True)
test_essays.head(10)

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
id,,,,,,,,,,,,,,,,,,,,,
0000aaaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7334,2982,1834
1111bbbb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2982,1834,8724
2222cccc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1834,8724,6133


In [9]:
seed = np.random.seed(6)

X = train_essays.drop(["generated"], axis= 1)
y = train_essays["generated"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = seed)

In [10]:
lgbmmodel = LGBMClassifier(random_state=seed)
lgbmmodel.fit(X,y)
print("CV score of LGBM is ",cross_val_score(lgbmmodel,X,y,cv=4, scoring = 'roc_auc').mean())

CV score of LGBM is  0.8517926566808751


In [11]:
xgbmodel = XGBClassifier(random_state=seed)
xgbmodel.fit(X,y)
print("CV score of XGB is ",cross_val_score(xgbmodel,X,y,cv=4, scoring = 'roc_auc').mean())

CV score of XGB is  0.8516836428006153


In [12]:
preds_xgb = xgbmodel.predict_proba(test_essays)[:,1]
preds_lgbm = lgbmmodel.predict_proba(test_essays)[:,1]
preds = (preds_xgb+preds_lgbm)/2

In [13]:
submission = pd.DataFrame()
submission["id"] = test_essays.index 
submission["generated"] = preds
submission.head(10)

,id,generated
0,0000aaaa,0.074375
1,1111bbbb,0.055892
2,2222cccc,0.063551
